In [17]:
trip_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet"
zone_lookup_url = "https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv"
# import pandas module to read the given URLs into dataframes
import pandas as pd
trip_df = pd.read_parquet(trip_url)
zone_lookup_df = pd.read_csv(zone_lookup_url)

In [18]:
# Perform data cleaning on trip_df. Drop missing values in the dataframe. Remove invalid values in the tip_amount and fare_amount columns
trip_df = trip_df.dropna()
trip_df = trip_df[(trip_df.tip_amount > 0) & (trip_df.fare_amount > 0)]

In [19]:
# Calculate the tip percentage by dividing the tip_amount by total_amount
trip_df['tip_percentage'] = (trip_df['tip_amount'] / trip_df['total_amount']) * 100

In [20]:
# Merge the two DataFrames on the columns PULocationID and LocationID. Name it as trip_df
trip_df = trip_df.merge(zone_lookup_df, left_on='PULocationID', right_on='LocationID')

In [21]:
# Perform one-hot encoding on the following categorical columns: VendorID, Borough, store_and_fwd_flag, and payment_type
trip_df = pd.get_dummies(trip_df, columns=['VendorID', 'Borough', 'store_and_fwd_flag', 'payment_type'])

In [22]:
# Drop the following columns from the preprocessed DataFrame: tpep_pickup_datetime, tpep_dropoff_datetime, Zone, PULocationID, DOLocationID
trip_df = trip_df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'Zone', 'PULocationID', 'DOLocationID', 'tip_amount', 'total_amount'], axis=1)

In [ ]:
# Partition trip_df into training, validation, and test sets and save it to s3://q-workshop-<account-id>/lab4/tlc/outputs as CSV files using AWS SDK for S3
from sklearn.model_selection import train_test_split
train_df, val_test_df = train_test_split(trip_df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(val_test_df, test_size=0.5, random_state=42)
import boto3
s3 = boto3.client('s3')
s3.put_object(Body=train_df.to_csv(index=False), Bucket='q-workshop-<account-ID>', Key='lab4/tlc/output/train.csv')
s3.put_object(Body=val_df.to_csv(index=False), Bucket='q-workshop-<account-ID>', Key='lab4/tlc/output/validation.csv')
s3.put_object(Body=test_df.to_csv(index=False), Bucket='q-workshop-<account-ID>', Key='lab4/tlc/output/test.csv')

In [24]:
test_2_df = test_df.drop(['tip_percentage'], axis=1)

In [ ]:
s3.put_object(Body=test_2_df.to_csv(index=False), Bucket='q-workshop-<account-ID>', Key='lab4/tlc/output/test_2.csv')